In [0]:
from pyspark.sql import SparkSession
spark

In [0]:
#1. Ingest the CSV files into two PySpark DataFrames
# Read customers.csv
df_customers = spark.read.option("header", True).option("inferSchema", True).csv("file:/Workspace/Shared/customers.csv")

# Read orders.csv
df_orders = spark.read.option("header", True).option("inferSchema", True).csv("file:/Workspace/Shared/orders.csv")

In [0]:
#2. Infer schema and print the schema for both
# Schema for customers
df_customers.printSchema()

# Schema for orders
df_orders.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Age: integer (nullable = true)

root
 |-- OrderID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- OrderDate: date (nullable = true)



In [0]:
#3. Add a column TotalAmount = Quantity * Price to orders
from pyspark.sql.functions import col

df_orders = df_orders.withColumn("TotalAmount", col("Quantity") * col("Price"))
df_orders.show()

+-------+----------+-------+--------+-----+----------+-----------+
|OrderID|CustomerID|Product|Quantity|Price| OrderDate|TotalAmount|
+-------+----------+-------+--------+-----+----------+-----------+
|   1001|       101| Laptop|       1|70000|2024-01-05|      70000|
|   1002|       102| Mobile|       2|25000|2024-02-10|      50000|
|   1003|       103|   Desk|       1|10000|2024-03-15|      10000|
|   1004|       101|  Mouse|       3| 1000|2024-04-01|       3000|
|   1005|       104|Monitor|       1|12000|2024-04-25|      12000|
+-------+----------+-------+--------+-----+----------+-----------+



In [0]:
#4. Join both DataFrames on CustomerID
df_joined = df_orders.join(df_customers, on="CustomerID", how="inner")
df_joined.show()

+----------+-------+-------+--------+-----+----------+-----------+-----+---------+---+
|CustomerID|OrderID|Product|Quantity|Price| OrderDate|TotalAmount| Name|     City|Age|
+----------+-------+-------+--------+-----+----------+-----------+-----+---------+---+
|       101|   1001| Laptop|       1|70000|2024-01-05|      70000|Aditi|   Mumbai| 28|
|       102|   1002| Mobile|       2|25000|2024-02-10|      50000|Rohan|    Delhi| 35|
|       103|   1003|   Desk|       1|10000|2024-03-15|      10000|Meena|Bangalore| 41|
|       101|   1004|  Mouse|       3| 1000|2024-04-01|       3000|Aditi|   Mumbai| 28|
|       104|   1005|Monitor|       1|12000|2024-04-25|      12000|Kabir|Hyderabad| 30|
+----------+-------+-------+--------+-----+----------+-----------+-----+---------+---+



In [0]:
#5. Filter orders where TotalAmount > 20000
df_filtered = df_joined.filter(col("TotalAmount") > 20000)
df_filtered.show()

+----------+-------+-------+--------+-----+----------+-----------+-----+------+---+
|CustomerID|OrderID|Product|Quantity|Price| OrderDate|TotalAmount| Name|  City|Age|
+----------+-------+-------+--------+-----+----------+-----------+-----+------+---+
|       101|   1001| Laptop|       1|70000|2024-01-05|      70000|Aditi|Mumbai| 28|
|       102|   1002| Mobile|       2|25000|2024-02-10|      50000|Rohan| Delhi| 35|
+----------+-------+-------+--------+-----+----------+-----------+-----+------+---+



In [0]:
#6. Show customers who placed more than 1 order
from pyspark.sql.functions import count

df_order_count = df_orders.groupBy("CustomerID").agg(count("*").alias("OrderCount")).filter("OrderCount > 1")
df_order_count.show()

+----------+----------+
|CustomerID|OrderCount|
+----------+----------+
|       101|         2|
+----------+----------+



In [0]:
#7. Group orders by City and get average order value
from pyspark.sql.functions import avg

df_avg_city = df_joined.groupBy("City").agg(avg("TotalAmount").alias("AverageOrderValue"))
df_avg_city.show()

+---------+-----------------+
|     City|AverageOrderValue|
+---------+-----------------+
|Bangalore|          10000.0|
|   Mumbai|          36500.0|
|    Delhi|          50000.0|
|Hyderabad|          12000.0|
+---------+-----------------+



In [0]:
#8. Sort orders by OrderDate in descending order
df_sorted = df_joined.orderBy(col("OrderDate").desc())
df_sorted.show()

+----------+-------+-------+--------+-----+----------+-----------+-----+---------+---+
|CustomerID|OrderID|Product|Quantity|Price| OrderDate|TotalAmount| Name|     City|Age|
+----------+-------+-------+--------+-----+----------+-----------+-----+---------+---+
|       104|   1005|Monitor|       1|12000|2024-04-25|      12000|Kabir|Hyderabad| 30|
|       101|   1004|  Mouse|       3| 1000|2024-04-01|       3000|Aditi|   Mumbai| 28|
|       103|   1003|   Desk|       1|10000|2024-03-15|      10000|Meena|Bangalore| 41|
|       102|   1002| Mobile|       2|25000|2024-02-10|      50000|Rohan|    Delhi| 35|
|       101|   1001| Laptop|       1|70000|2024-01-05|      70000|Aditi|   Mumbai| 28|
+----------+-------+-------+--------+-----+----------+-----------+-----+---------+---+



In [0]:
#9. Write the final result as a Parquet file partitioned by City
df_joined.write.mode("overwrite").partitionBy("City").parquet("/dbfs/FileStore/orders_partitioned_by_city")


In [0]:
#10. Create a temporary view and run Spark SQL
# Register temp view
df_joined.createOrReplaceTempView("orders_view")

In [0]:
#Total sales by customer
spark.sql("""
SELECT Name, SUM(TotalAmount) AS TotalSales
FROM orders_view
GROUP BY Name
""").show()

+-----+----------+
| Name|TotalSales|
+-----+----------+
|Kabir|     12000|
|Rohan|     50000|
|Aditi|     73000|
|Meena|     10000|
+-----+----------+



In [0]:
#Count of products per city
spark.sql("""
SELECT City, COUNT(Product) AS ProductCount
FROM orders_view
GROUP BY City
""").show()

+---------+------------+
|     City|ProductCount|
+---------+------------+
|Bangalore|           1|
|   Mumbai|           2|
|    Delhi|           1|
|Hyderabad|           1|
+---------+------------+



In [0]:
#Top 2 cities by revenue
spark.sql("""
SELECT City, SUM(TotalAmount) AS Revenue
FROM orders_view
GROUP BY City
ORDER BY Revenue DESC
LIMIT 2""").show()


+------+-------+
|  City|Revenue|
+------+-------+
|Mumbai|  73000|
| Delhi|  50000|
+------+-------+

